In [1]:
BIZPATH='../../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json'
USERPATH='../../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_user.json'
REVPATH='../../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'
PARTIAL_DIM_PATH = '../full_dataset/featureset1_resto_food_chinese.json'

In [2]:
import json

In [3]:
bizrecords = [json.loads(line) for line in open(BIZPATH)]
userrecords = [json.loads(line) for line in open(USERPATH)]
revrecords = [json.loads(line) for line in open(REVPATH)]
pdimrecords = [json.loads(line) for line in open(PARTIAL_DIM_PATH)]

In [4]:
from pandas import DataFrame, Series

In [5]:
import pandas as pd; import numpy as np

In [6]:
bizframe = DataFrame(bizrecords)
userframe = DataFrame(userrecords)
revframe = DataFrame(revrecords)
pdimframe = pd.read_json(PARTIAL_DIM_PATH)
pdimframe.head()

,categories_avg_rating,categories_biz_count,categories_reviews,std_dev_rating,total_reviews,user_id
0,5,1,1,0,4,-BjekrEUhweLVyrjVStMgg
1,5,1,1,0,6,XQiaNhoZbdxaN3pJIBADvw
10,2,1,1,0,12,82d-mm3hB2UelHwrptvcMw
100,2,1,1,0,22,0XMC32Lkh6x1Yc9m9sSK0Q
1000,1,1,1,0,1,_j907SJO1Tzl102sXiJPLA


In [7]:
bizframe_sub = DataFrame(bizrecords, columns=['business_id', 'name', 'categories', 'review_count', 'stars'])

In [8]:
rev_user_frame = pd.merge(revframe, userframe, on="user_id")
rev_biz_frame = pd.merge(revframe, bizframe, on="business_id")

In [9]:
#Months yelping
import datetime
now = datetime.datetime.now()
userframe['months_yelping'] = userframe['yelping_since'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m')).map(lambda x: (now.year - x.year) * 12 + now.month - x.month)

In [10]:
userframe.head()

,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since,months_yelping
0,4.14,"{u'profile': 8, u'cute': 15, u'funny': 11, u'p...","[2005, 2006]",70,"[rpOyqD_893cqmDAtJLbdog, 4U9kSBLuBDU391x6bxU-Y...",Russel,108,user,18kPq7GPye-YQ3LyKyAZPw,"{u'funny': 167, u'useful': 280, u'cool': 245}",2004-10,137
1,3.66,"{u'profile': 116, u'cute': 206, u'funny': 580,...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",1179,"[18kPq7GPye-YQ3LyKyAZPw, 4U9kSBLuBDU391x6bxU-Y...",Jeremy,1274,user,rpOyqD_893cqmDAtJLbdog,"{u'funny': 7681, u'useful': 14199, u'cool': 11...",2004-10,137
2,3.60,"{u'profile': 9, u'cute': 23, u'funny': 120, u'...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",100,"[18kPq7GPye-YQ3LyKyAZPw, rpOyqD_893cqmDAtJLbdo...",Michael,442,user,4U9kSBLuBDU391x6bxU-YA,"{u'funny': 908, u'useful': 1483, u'cool': 732}",2004-10,137
3,4.64,{u'cute': 2},[],2,"[18kPq7GPye-YQ3LyKyAZPw, rpOyqD_893cqmDAtJLbdo...",Ken,11,user,fHtTaujcyKvXglE33Z5yIw,"{u'funny': 1, u'useful': 11, u'cool': 5}",2004-10,137
4,3.80,"{u'cute': 2, u'plain': 2, u'writer': 2, u'note...",[2005],4,"[rpOyqD_893cqmDAtJLbdog, HDQixQ-WZEV0LVPJlIGQe...",Katherine,66,user,SIBCL7HBkrP4llolm4SC2A,"{u'funny': 11, u'useful': 34, u'cool': 13}",2004-10,137


In [11]:
#Label elite users
userframe["elite_user"] = userframe['elite'].map(lambda x: 'elite' if len(x) > 0 else 'not_elite')

In [12]:
#Adapted from: http://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
#TODO: Find more idiomatic definition for rev_category_frame, preferably by removing flattened_categories.
flattened_categories = DataFrame([[i, category]
                                     for i, categories in rev_biz_frame['categories'].apply(list).iteritems()
                                         for category in categories],
                                columns=['index', 'category'])
flattened_categories.set_index('index')
rev_category_frame = rev_biz_frame.merge(flattened_categories, left_index = True, right_index = True)

In [13]:
#Number businesses / category
#TODO: Add name to the size column; do some sanity checking on the data.
user_biz_revved_per_category_frame = rev_category_frame.groupby(['user_id', 'business_id', 'category']).size()

In [14]:
rev_restaurants_frame = DataFrame(rev_category_frame.loc[rev_category_frame['category'] == 'Restaurants'])
rev_restaurants_frame['funny'] = rev_restaurants_frame['votes'].map(lambda x: x['funny'])
rev_restaurants_frame['useful'] = rev_restaurants_frame['votes'].map(lambda x: x['useful'])
rev_restaurants_frame['cool'] = rev_restaurants_frame['votes'].map(lambda x: x['cool'])

In [15]:
rev_dim_peruser_frame = rev_restaurants_frame.groupby('user_id').aggregate(sum).reset_index()[['user_id', 'funny', 'useful', 'cool']]

In [16]:
user_dim_frame = pd.merge(userframe[['user_id', 'months_yelping', 'elite_user']], rev_dim_peruser_frame, on='user_id')
dimframe = pd.merge(pdimframe, user_dim_frame, on='user_id')

In [17]:
#Move elite_user column to end
elite_column = dimframe.pop('elite_user')
dimframe.insert(10,'elite_user', elite_column)

In [18]:
dimframe.to_json('../full_dataset/featureset1.json')

In [19]:
dimframe.head()

,categories_avg_rating,categories_biz_count,categories_reviews,std_dev_rating,total_reviews,user_id,months_yelping,funny,useful,cool,elite_user
0,5,1,1,0,4,-BjekrEUhweLVyrjVStMgg,17,0,0,0,not_elite
1,5,1,1,0,6,XQiaNhoZbdxaN3pJIBADvw,13,0,1,0,not_elite
2,2,1,1,0,12,82d-mm3hB2UelHwrptvcMw,137,0,0,0,not_elite
3,2,1,1,0,22,0XMC32Lkh6x1Yc9m9sSK0Q,62,0,0,1,not_elite
4,1,1,1,0,1,_j907SJO1Tzl102sXiJPLA,14,0,1,0,not_elite
